<a href="https://colab.research.google.com/github/Hao-191/hello-hao/blob/master/Lyrics_(nophoneme_Pretrained_Bert2GPT2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Environment


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%env HOME=/content/drive/My Drive/
%cd ~/
!mkdir -p Courses/nlp_spring2022/final/checkpoints
%cd Courses/nlp_spring2022/final

env: HOME=/content/drive/My Drive/
/content/drive/My Drive
/content/drive/My Drive/Courses/nlp_spring2022/final


In [ ]:
!nvidia-smi

Mon May 16 14:28:34 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install datasets nltk tensorboard keybert

!pip install transformers


  Using cached datasets-2.2.1-py3-none-any.whl (342 kB)
  Using cached keybert-0.5.1-py3-none-any.whl
  Using cached fsspec-2022.3.0-py3-none-any.whl (136 kB)
  Using cached aiohttp-3.8.1-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.1 MB)
     |████████████████████████████████| 84 kB 1.3 MB/s 
  Using cached responses-0.18.0-py3-none-any.whl (38 kB)
  Using cached xxhash-3.0.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (212 kB)
  Using cached urllib3-1.25.11-py2.py3-none-any.whl (127 kB)
  Using cached rich-12.4.1-py3-none-any.whl (231 kB)
  Using cached sentence_transformers-2.2.0-py3-none-any.whl
  Using cached commonmark-0.9.1-py2.py3-none-any.whl (51 kB)
     |████████████████████████████████| 4.2 MB 29.8 MB/s 
  Using cached sentencepiece-0.1.96-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (1.2 MB)
  Using cached tokenizers-0.12.1-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (6.6 MB)
  

In [ ]:
import random
import numpy as np
import os
import torch
import math
!date

Mon May 16 14:29:35 UTC 2022


#GPU State


In [ ]:
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

In [ ]:
import psutil
import humanize
import os
import GPUtil as GPU

GPUs = GPU.getGPUs()

gpu = GPUs[0]
def printm():
  process = psutil.Process(os.getpid())
  print("Gen RAM Free: " + humanize.naturalsize(psutil.virtual_memory().available), " |    Proc size: " + humanize.naturalsize(process.memory_info().rss))
  print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total    {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))

printm()

Gen RAM Free: 12.3 GB  |    Proc size: 331.5 MB
GPU RAM Free: 15109MB | Used: 0MB | Util   0% | Total    15109MB


In [ ]:
torch.cuda.empty_cache()

# Data Pre-processing

This part is used to pre-process the rap lyric texts that we downloaded from Kaggle. 

I am going to re-organize the texts into CSV files containing features of (artist_name, this_line,next_line, phonemes, keywords), and split them into train, dev, and test sets with a proportion of (7:1.5:1.5).

##Make .csv files

In [ ]:

import pandas as pd
import os
from sklearn.model_selection import train_test_split
import nltk
from keybert import KeyBERT

nltk.download('cmudict')
kw_model = KeyBERT(model='all-mpnet-base-v2')

'''
    Raw lyrics files downloaded from
    https://www.kaggle.com/datasets/rikdifos/rap-lyrics
    Shout out to the dataset builder
    Removed Tupac1_lyrics.txt and Tupac2_lyrics.txt and The Notorious Big_lyrics.txt
'''

# use cmudict to get the last 2 vowels of the sentence
def get_last_vowels(content,limit=2):
    l = []
    all_vowels = {"AA":0,"AE":1,"AH":2,"AO":3,"AW":4,"AY":5,"EH":6,"ER":7,"EY":8,"IH":9,"IY":10,"OW":11,"OY":12,"UH":13,"UW":14,"[PAD]":15}
    dic = nltk.corpus.cmudict.dict()
    for line in content:
        if line != '[PAD]':
            vowels = []
            words = line.split()[-5:]
            for i in range(len(words)-1,-1,-1):
                if len(vowels)>limit-1:
                    break
                try:
                    word = words[i].lower()
                    count = 0
                    while word not in dic.keys() and count < 2:
                        count += 1
                        word = word[:-1]
                    sounds = dic[word]
                    sounds = sounds[0]
                    for j in range(len(sounds)-1,-1,-1):
                        if len(vowels)>limit-1:
                            break
                        if sounds[j] in all_vowels.keys():
                            vowels.append(sounds[j])
                        elif sounds[j][:-1] in all_vowels.keys():
                            vowels.append(sounds[j][:-1])
                except:
                    continue
            while len(vowels) < limit:
                vowels.append("[PAD]")



        else:
            vowels = ["[PAD]"]*limit


        vowels.reverse()
        vowels_str = ','.join(vowels)
        l.append(vowels_str)

    return l

##Use KeyBERT model to extract 3 keywords from a verse
def get_keywords(content,model,n = 10):
    kwds = []
    this_verse = ''
    new_verse = False
    line_count = 0
    for i in range(len(content)-1):
        this_line = content[i]
        next_line = content[i+1]
        if (this_line=='[PAD]') and (next_line!="[PAD]"):
            new_verse = True
            this_verse = ''
            line_count = 0
            kwds += ["[PAD]"]
        elif (this_line == '[PAD]') and (next_line=='[PAD]'):
            this_verse = ''
            line_count = 0
            kwds += ["[PAD]"]
        elif (this_line!='[PAD]') and (next_line=='[PAD]'):
            new_verse = False
            this_verse += this_line
            line_count += 1
            kwd = model.extract_keywords(this_verse,keyphrase_ngram_range=(1,1),stop_words='english',highlight=False,top_n=n)
            kwd_l = [item[0] for item in kwd if item[0] not in ["nigga","niggas","bitch","fuck","pussy","bitches","bitchin"]][:3]
            kwd_s = ",".join(kwd_l)
            if len(kwd_s) < 3:
                kwd_s = "[PAD]"
            kwds += [kwd_s for i in range(line_count)]
        elif (this_line!='[PAD]') and (next_line != '[PAD]'):
            this_verse += this_line
            line_count += 1

    kwds += ["[PAD]"]
    return kwds

# convert all .txt files to a csv file with columns (this_line, next_line)
def create_csv_file(kw_model):
    

    data = {}
    frames = []
    phonemes = []
    files = []
    for (dirpath, dirnames, filenames) in os.walk('./data/lyrics/'):
        files.extend(filenames)

    for filename in files:
        artist_name = filename[:-11]
        print("Starting with " + artist_name + "...")
        if filename[-4:] == '.txt':
            with open("./data/lyrics/"+filename,'r') as f:
                content = f.read().splitlines()

                ##Clean the "(" ")"...
                for i in range(len(content)-1,-1,-1):
                    no_meaning = False
                    seq = content[i]
                    seq = seq.split()
                    for j in range(len(seq)):
                        if ("(" in seq[j]) or (")" in seq[j]):
                            no_meaning = True
                    if no_meaning:
                        content.pop(i)


                for i in range(len(content)-1,-1,-1):
                    if len(content[i].split()) <= 5 and content[i]:
                        content.pop(i)
                
                for i in range(len(content)):
                    if content[i] == '':
                        content[i] = '[PAD]'
                content.append('[PAD]')

                ##Getting the phonemetic features for last word of each line
                phonemes = get_last_vowels(content)
            
                ##Getting keyword for each verse. All lines in the same verse will receive same keywords
                keywords = get_keywords(content,kw_model)


                this_line_cleaned = content[:-1]
                next_line_cleaned = content[1:]
                phonemes_cleaned = phonemes[:-1]
                keyword_cleaned = keywords[:-1]

                ##Remove the lines that is empty
                for i in range(len(this_line_cleaned)-1,-1,-1):
                  if this_line_cleaned[i] == '[PAD]' or next_line_cleaned[i] == '[PAD]':
                    this_line_cleaned.pop(i)
                    next_line_cleaned.pop(i)
                    phonemes_cleaned.pop(i)
                    keyword_cleaned.pop(i)


                ##Clean Repetitive lytics 
                for i in range(len(this_line_cleaned)-1,-1,-1):
                  if this_line_cleaned[i].split()[:5] == next_line_cleaned[i].split()[:5]:
                    this_line_cleaned.pop(i)
                    next_line_cleaned.pop(i)
                    phonemes_cleaned.pop(i)
                    keyword_cleaned.pop(i)

                data['artist_name'] = artist_name
                data['this_line'] = this_line_cleaned
                data['next_line'] = next_line_cleaned
                data['phonemes'] = phonemes_cleaned
                data['keywords'] = keyword_cleaned

           

                df_temp = pd.DataFrame(data)
                frames.append(df_temp)
    df = pd.concat(frames)
    df.to_csv("./data/data_csv/lyrics_ult.csv")
    # print(df.head()['this_line'])
    #Splitting the dataframes into train, dev and test (7:1.5:1.5)
    train,test_and_dev = train_test_split(df,test_size = 0.3)
    dev,test = train_test_split(test_and_dev,test_size = 0.5)

    print(train.size,dev.size,test.size)
    #Save them to csv
    train.to_csv("./data/data_csv/train_ult.csv")
    dev.to_csv("./data/data_csv/valid_ult.csv")
    test.to_csv("./data/data_csv/test_ult.csv")



In [ ]:
#create_csv_file(kw_model)


## Setup

In [ ]:
import random
import torch

gradient_accumulation_steps = 1
learning_rate = 6.25e-5
adam_epsilon = 1e-8
max_grad_norm = 1.0
num_train_epochs = 10
warmup_steps = 0

seed = 42
batch_size = 8
max_seq_length = 384
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
sentence_encoder_model_name = "bert-base-uncased"
keyword_encoder_model_name = "bert-base-uncased"
decoder_model_name = "distilgpt2"

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

ph_dic = {"AA":0,"AE":1,"AH":2,"AO":3,"AW":4,"AY":5,"EH":6,"ER":7,"EY":8,"IH":9,"IY":10,"OW":11,"OY":12,"UH":13,"UW":14,"[PAD]":15}
def ph_decode(l,ph_dic):
  l2 = []
  for i in l:
    for k,v in ph_dic.items():
      if v == i:
        l2.append(k)
        break
  return ",".join(l2)

## Tokenizer

Like in Question 1, we will use the GPT2 tokenizer to tokenize text. Any new word not in the GPT2 vocabulary will be split into subwords automatically.

### Config for Models


In [ ]:
from transformers import AutoConfig, AutoTokenizer

sentence_encoder_config = AutoConfig.from_pretrained(sentence_encoder_model_name)
keyword_encoder_config = AutoConfig.from_pretrained(keyword_encoder_model_name)
decoder_config = AutoConfig.from_pretrained(decoder_model_name)

# sentence_encoder_config.vocab_size = 50257
# keyword_encoder_config.vocab_size = 50257
# decoder_config.vocab_size = 50257
decoder_config.layer_norm_eps = 1e-12
decoder_config.hidden_dropout_prob = 0.1
decoder_config.attention_probs_dropout_prob = 0.1
decoder_config.intermediate_size = 768
decoder_config.hidden_act = "gelu"
decoder_config.pad_token_id=0

In [ ]:

sentence_encoder_tokenizer = AutoTokenizer.from_pretrained(sentence_encoder_model_name)
keyword_encoder_tokenizer = AutoTokenizer.from_pretrained(keyword_encoder_model_name)
decoder_tokenizer = AutoTokenizer.from_pretrained(decoder_model_name)


# sentence_encoder_tokenizer.add_special_tokens({'pad_token': '[PAD]'})
# sentence_encoder_tokenizer.add_special_tokens({'sep_token': '[SEP]'})
# #sentence_encoder_tokenizer.add_tokens(['[Empty]'],special_tokens = True)

# keyword_encoder_tokenizer.add_special_tokens({'pad_token': '[PAD]'})
# keyword_encoder_tokenizer.add_special_tokens({'sep_token': '[SEP]'})
# #keyword_encoder_tokenizer.add_tokens(['[Empty]'],special_tokens = True)

# decoder_tokenizer.add_special_tokens({'pad_token': '[PAD]'})
# decoder_tokenizer.add_special_tokens({'sep_token': '[SEP]'})
# #decoder_tokenizer.add_tokens(['[Empty]'],special_tokens = True)
decoder_tokenizer.pad_token = decoder_tokenizer.eos_token


## Dataset

Let's use the bill summarization corpus for our task. For simplicity, we only concern how to generate the title of a bill article.

In [ ]:
import datasets

train_dataset = datasets.load_dataset('csv', data_files = "./data/data_csv/train_ult.csv")['train']
valid_dataset = datasets.load_dataset('csv', data_files = "./data/data_csv/valid_ult.csv")['train']
test_dataset = datasets.load_dataset('csv', data_files = "./data/data_csv/test_ult.csv")['train']


Using custom data configuration default-f6d6e6b9d8a57214
Reusing dataset csv (/content/drive/My Drive/.cache/huggingface/datasets/csv/default-f6d6e6b9d8a57214/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-9e1e0dfc39dccec1
Reusing dataset csv (/content/drive/My Drive/.cache/huggingface/datasets/csv/default-9e1e0dfc39dccec1/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-2b8af1ed0e9e49ca
Reusing dataset csv (/content/drive/My Drive/.cache/huggingface/datasets/csv/default-2b8af1ed0e9e49ca/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
print(train_dataset)
print(valid_dataset)
print(test_dataset)

Dataset({
    features: ['Unnamed: 0', 'artist_name', 'this_line', 'next_line', 'phonemes', 'keywords'],
    num_rows: 69955
})
Dataset({
    features: ['Unnamed: 0', 'artist_name', 'this_line', 'next_line', 'phonemes', 'keywords'],
    num_rows: 14991
})
Dataset({
    features: ['Unnamed: 0', 'artist_name', 'this_line', 'next_line', 'phonemes', 'keywords'],
    num_rows: 14991
})


In [ ]:
x = 200
print(len(train_dataset['artist_name'][x].split()))
print(len(train_dataset['this_line'][x].split()))
print(len(train_dataset['next_line'][x].split()))
print(len(train_dataset['phonemes'][x].split()))
print(train_dataset['this_line'][x])
print(train_dataset['next_line'][x])
print(train_dataset['phonemes'][x])
print(train_dataset['keywords'][x])

2
9
10
1
Pop a shot, kill 'em all, kill 'em all
It's just me and my niggas taking over the game
AH,AO
rappers,shooters,tryna


### Implement `class RapDataset`

* Implement a function to prepare input and target pairs
* Implement a function to pad a batch of samples that have different lengths.
* Implement a function to return a batch of tensors of 
 padded input_ids and lm_labels.

In [ ]:
import re
import torch
from torch.utils.data import Dataset



class RapDataset(Dataset):
  """
  This class will prepare a batch of sequences that are properly padded
  due to different lengths in each sentence in a batch.
  """

  def __init__(self, datasets,se_tokenizer, ke_tokenizer,d_tokenizer, keywords_column_name = "keywords",phonemes_column_name="phonemes",this_line_column_name="this_line", next_line_column_name='next_line', artist_column_name = 'artist_name', max_length=384):
    self._this_line_column_name = this_line_column_name
    self._next_line_column_name = next_line_column_name
    self._artist_column_name = artist_column_name
    self._phonemes_column_name = phonemes_column_name
    self._keywords_column_name = keywords_column_name
    self._preprocessing_num_workers = 4
    self._overwrite_cache = False
    self._padding = False
    self._datasets = datasets
    self._se_tokenizer = se_tokenizer
    self._ke_tokenizer = ke_tokenizer
    self._d_tokenizer = d_tokenizer
    self._ph_dic = {"AA":0,"AE":1,"AH":2,"AO":3,"AW":4,"AY":5,"EH":6,"ER":7,"EY":8,"IH":9,"IY":10,"OW":11,"OY":12,"UH":13,"UW":14,"[PAD]":15}

    self._d_bos = d_tokenizer.bos_token_id
    self._d_eos = d_tokenizer.eos_token_id

    self._se_sep = se_tokenizer.sep_token_id
    self._se_cls = se_tokenizer.cls_token_id
    self._ke_sep = ke_tokenizer.sep_token_id
    self._ke_cls = ke_tokenizer.cls_token_id

    self._max_length = max_length

    # tokenize the raw text, creating a new field "input_ids"
    self._tokenized_datasets = datasets.map(
      self._tokenize,
      batched=True,
      num_proc=self._preprocessing_num_workers,
      remove_columns=[],
      load_from_cache_file=not self._overwrite_cache,
    )

  def _clean(self, text):
    if isinstance(text, list):
      result = []
      for x in text:
        xx = re.sub(r'\\n', ' ', x)
        xx = re.sub(r'\s+', ' ', xx)
        result.append(xx)
      return result
    else:
      xx = re.sub(r'\\n', ' ', text)
      xx = re.sub(r'\s+', ' ', xx)
      return xx

  def _tokenize(self, example):
    this_line_output = self._se_tokenizer(self._clean(example[self._this_line_column_name]), padding=self._padding, truncation=True, max_length=self._max_length)
    next_line_output = self._d_tokenizer(self._clean(example[self._next_line_column_name]), padding=self._padding, truncation=True, max_length=self._max_length)
    keyword_output = self._ke_tokenizer(self._clean(example[self._keywords_column_name]), padding=self._padding, truncation=True, max_length=self._max_length)
    keyword_ids = keyword_output['input_ids']

    phonemes_l = example[self._phonemes_column_name]
    phoneme_ids = []
    for string in phonemes_l:
      if string:
        l = string.split(",")
        for i in range(len(l)):
          l[i] = self._ph_dic[l[i]]
        phoneme_ids.append(l)
      

    return {'this_ids': this_line_output['input_ids'],
            'next_ids': next_line_output['input_ids'],
            'artist_names': example[self._artist_column_name],
            'phoneme_ids': phoneme_ids,
            'keyword_ids':keyword_ids
           }
  
  def _build_input_from_segments(self, this_line, next_line,artist_name,phoneme_ids,keyword_ids):
    instance = {}
    instance['input_ids'] = [self._d_bos] + next_line 
    instance['lm_labels'] = next_line + [self._d_eos]
    instance['phoneme_ids'] = phoneme_ids
    instance['this_line'] = this_line
    instance['next_line'] = next_line
    instance['artist_names'] = artist_name
    instance['keyword_ids'] = keyword_ids

    return instance

  def __len__(self):
    return len(self._tokenized_datasets)

  def __getitem__(self, index):
    item = self._tokenized_datasets[index]
    return self._build_input_from_segments(item['this_ids'], item['next_ids'],item['artist_names'],item['phoneme_ids'],item['keyword_ids'])

  def _pad_ids(self, arrays, padding, max_length=-1):
    if max_length == -1:
      length = max([len(i) for i in arrays]) 
    else:
      length = max([len(i) for i in arrays] + [max_length])

    for i in range(len(arrays)):
      arrays[i] = arrays[i] + [padding for i in range(length - len(arrays[i]))]


    return arrays



  def collate_fn(self, batch):
    """
    batch: It should contain a list of Python dictionary returned from __getitem__
    """
    input_ids = [seq['input_ids'] for seq in batch]
    lm_labels = [seq['lm_labels'] for seq in batch]
    phoneme_ids = [seq['phoneme_ids'] for seq in batch]
    artist_name_ids = [seq['artist_names'] for seq in batch]
    keyword_ids = [seq['keyword_ids'] for seq in batch]
    this_line_ids = [seq['this_line'] for seq in batch]

    imput_ids = self._pad_ids(input_ids,self._d_tokenizer.pad_token_id)
    lm_labels = self._pad_ids(lm_labels,self._d_tokenizer.pad_token_id)
    keywords_ids = self._pad_ids(keyword_ids,self._ke_tokenizer.pad_token_id)
    this_line_ids = self._pad_ids(this_line_ids,self._se_tokenizer.pad_token_id)


    input_ids = torch.from_numpy(np.asarray(input_ids))
    lm_labels = torch.from_numpy(np.asarray(lm_labels))
    phoneme_ids =  torch.from_numpy(np.asarray(phoneme_ids))
    keyword_ids = torch.from_numpy(np.asarray(keyword_ids))
    this_line_ids = torch.from_numpy(np.asarray(this_line_ids))
    # artist_name_ids = torch.from_numpy(np.asarray(artist_name_ids))
    return input_ids.to(device), lm_labels.to(device), phoneme_ids.to(device), keyword_ids.to(device), this_line_ids.to(device)

In [ ]:
train_rap_dataset = RapDataset(train_dataset,sentence_encoder_tokenizer,keyword_encoder_tokenizer,decoder_tokenizer)
valid_rap_dataset = RapDataset(valid_dataset,sentence_encoder_tokenizer,keyword_encoder_tokenizer,decoder_tokenizer)
test_rap_dataset = RapDataset(test_dataset,sentence_encoder_tokenizer,keyword_encoder_tokenizer,decoder_tokenizer)

Loading cached processed dataset at /content/drive/My Drive/.cache/huggingface/datasets/csv/default-f6d6e6b9d8a57214/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-91591516abba3625.arrow


Loading cached processed dataset at /content/drive/My Drive/.cache/huggingface/datasets/csv/default-f6d6e6b9d8a57214/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-46f245440b1904e0.arrow


Loading cached processed dataset at /content/drive/My Drive/.cache/huggingface/datasets/csv/default-f6d6e6b9d8a57214/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-77ea6b7433f5792d.arrow


Loading cached processed dataset at /content/drive/My Drive/.cache/huggingface/datasets/csv/default-f6d6e6b9d8a57214/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-f89601c3084be338.arrow


Loading cached processed dataset at /content/drive/My Drive/.cache/huggingface/datasets/csv/default-9e1e0dfc39dccec1/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-4a6905e854ac3f84.arrow


...

Loading cached processed dataset at /content/drive/My Drive/.cache/huggingface/datasets/csv/default-9e1e0dfc39dccec1/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-5d127167312f752a.arrow


Loading cached processed dataset at /content/drive/My Drive/.cache/huggingface/datasets/csv/default-2b8af1ed0e9e49ca/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-ac5027c302778d55.arrow


Loading cached processed dataset at /content/drive/My Drive/.cache/huggingface/datasets/csv/default-2b8af1ed0e9e49ca/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-300599d5f2fb268f.arrow


Loading cached processed dataset at /content/drive/My Drive/.cache/huggingface/datasets/csv/default-2b8af1ed0e9e49ca/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-0cb59834b17ba4eb.arrow


Loading cached processed dataset at /content/drive/My Drive/.cache/huggingface/datasets/csv/default-2b8af1ed0e9e49ca/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519/cache-1498ae669fd9c9f4.arrow


In [ ]:
x = 20

print("Input IDs for Sentence Encoder:\n\t", train_rap_dataset[x]['this_line'])
print("Input IDs for Keyword Encoder:\n \t",train_rap_dataset[x]['keyword_ids'])
print("Input IDs for Phoneme Encoder:\n\t",train_rap_dataset[x]['phoneme_ids'])
print("Input IDs for Decoder:\n\t", train_rap_dataset[x]['input_ids'])
print("LM Labels for Decoder:\n\t",train_rap_dataset[x]['lm_labels'])

print("Input sentence for Sentence Encoder:\n\t",sentence_encoder_tokenizer.decode(train_rap_dataset[x]['this_line']))
print("Input keywords for Keyword Encoder:\n\t",keyword_encoder_tokenizer.decode(train_rap_dataset[x]['keyword_ids']))
print("Input keywords for Phoneme Encoder:\n\t",ph_decode(train_rap_dataset[x]['phoneme_ids'],ph_dic))
print("Input sentence for Decoder:\n\t", decoder_tokenizer.decode(train_rap_dataset[x]['input_ids']))
print("LM Ground Truth Sentence for Decoder:\n\t",decoder_tokenizer.decode(train_rap_dataset[x]['lm_labels']))


Input IDs for Sentence Encoder:
	 [101, 2330, 2039, 1996, 11002, 1010, 2016, 1005, 1055, 2074, 3564, 2045, 1010, 7592, 102]
Input IDs for Keyword Encoder:
 	 [101, 7133, 1010, 10181, 1010, 3566, 102]
Input IDs for Phoneme Encoder:
	 [2, 11]
Input IDs for Decoder:
	 [50256, 5122, 5273, 338, 4506, 11, 3521, 470, 772, 787, 4151, 2800, 618, 356, 2740]
LM Labels for Decoder:
	 [5122, 5273, 338, 4506, 11, 3521, 470, 772, 787, 4151, 2800, 618, 356, 2740, 50256]
Input sentence for Sentence Encoder:
	 [CLS] open up the curtain, she's just sitting there, hello [SEP]
Input keywords for Keyword Encoder:
	 [CLS] grandmother, deceased, mom [SEP]
Input keywords for Phoneme Encoder:
	 AH,OW
Input sentence for Decoder:
	 <|endoftext|>Our conversation's brief, couldn't even make eye contact when we speak
LM Ground Truth Sentence for Decoder:
	 Our conversation's brief, couldn't even make eye contact when we speak<|endoftext|>


In [ ]:
# Create dataloader for traininz
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

print(batch_size)
train_dataloader = DataLoader(
  train_rap_dataset,
  sampler=RandomSampler(train_rap_dataset),
  batch_size=batch_size,
  collate_fn=train_rap_dataset.collate_fn
)

valid_dataloader = DataLoader(
  valid_rap_dataset,
  sampler=SequentialSampler(valid_rap_dataset),
  batch_size=batch_size,
  collate_fn=valid_rap_dataset.collate_fn
)
test_dataloader = DataLoader(
  test_rap_dataset,
  sampler=SequentialSampler(test_rap_dataset),
  batch_size=batch_size,
  collate_fn=test_rap_dataset.collate_fn
)


x = 2
for i, batch in enumerate(train_dataloader):
  input_ids, lm_labels,phoneme_ids,keyword_ids,this_line_ids = batch
  print(len(this_line_ids[x]),len(input_ids[x]), len(lm_labels[x]),len(phoneme_ids[x]),len(keyword_ids[x]))
  print("Padded Input IDs of Sentence Encoder: \n\t",this_line_ids[x])
  print("Padded Input IDs of Phoneme Encoder: \n\t",phoneme_ids[x])
  print("Padded Input IDs of Keyword Encoder: \n\t",keyword_ids[x])
  print("Padded Input IDs of Decoder: \n\t",input_ids[x])
  print("Padded Ground Truth IDs of Decoder: \n\t",lm_labels[x])
  print()


  if i >= 0:
    break

8
19 14 14 2 9
Padded Input IDs of Sentence Encoder: 
	 tensor([ 101, 2057, 7110, 1005, 1056, 1999, 1037, 9241, 1010, 5481, 2378, 1005,
        2046, 2293,  102,    0,    0,    0,    0], device='cuda:0')
Padded Input IDs of Phoneme Encoder: 
	 tensor([14,  2], device='cuda:0')
Padded Input IDs of Keyword Encoder: 
	 tensor([  101,  4184,  4402,  1010, 12925, 28345,  1010, 24438,   102],
       device='cuda:0')
Padded Input IDs of Decoder: 
	 tensor([50256,  8061, 11892,   777, 40207,    11,   705, 47163,   484,  1282,
          287,  2800,   351, 15391], device='cuda:0')
Padded Ground Truth IDs of Decoder: 
	 tensor([ 8061, 11892,   777, 40207,    11,   705, 47163,   484,  1282,   287,
         2800,   351, 15391, 50256], device='cuda:0')



Data is completely pre-processed above. 

# Model

##Initialize sub pretrained models


In [ ]:
from transformers import EncoderDecoderModel, AutoModel
k_encoder_name = 'bert-base-uncased'
k_encoder = AutoModel.from_pretrained(k_encoder_name)
k_encoder.to(device)
encoder_name = 'bert-base-uncased'
decoder_name = 'distilgpt2'
bert2gpt2 = EncoderDecoderModel.from_encoder_decoder_pretrained(encoder_name, decoder_name,  tie_encoder_decoder=True)
bert2gpt2.to(device)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_

EncoderDecoderModel(
  (encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_af

##Design Encoder Decoder Model with all features



In [ ]:
import torch.nn as nn
class Encoder_Decoder_LM(nn.Module):
  def __init__(self,k_encoder,bert2gpt2,s_tokenizer,k_tokenizer,d_tokenizer):
    super().__init__()

    # self.p_emb = nn.Embedding(16,768)
    # self.p_layer = nn.Linear(768,768)
    self.w_k = nn.Linear(768,768)


    self.k_encoder = k_encoder
    self.bert2gpt2 = bert2gpt2
    self.s_tok = s_tokenizer
    self.k_tok = k_tokenizer
    self.d_tok = d_tokenizer

    

  def make_attention_mask(self,ids,tok):
    return torch.log(torch.ne(ids,tok.pad_token_id))

  def forward(self,this_line_ids, keyword_ids, input_ids,phoneme_ids):
    #getting tensors' sizes
    B = this_line_ids.size()[0]
    M = this_line_ids.size()[1]
    


    ##making masks
    s_mask = self.make_attention_mask(this_line_ids,self.s_tok)
    k_mask = self.make_attention_mask(keyword_ids,self.k_tok)
    d_mask = self.make_attention_mask(input_ids,self.d_tok)
    
    ##getting hidden states
    k_hidden_cls = self.k_encoder(keyword_ids,attention_mask=k_mask).last_hidden_state[:,0,:][:,None,]
    hidden_state= self.bert2gpt2(input_ids=this_line_ids,attention_mask=s_mask, 
                        decoder_input_ids = input_ids, decoder_attention_mask = d_mask).encoder_last_hidden_state
    # phoneme_embs = self.p_emb(phoneme_ids)
    # p_hidden = self.p_layer(phoneme_embs)

    #repeating cls hidden states for M tokens
    k_hidden_cls = k_hidden_cls.repeat(1,M,1)
    k_hidden_cls = self.w_k(k_hidden_cls)

    #Create phoneme feature tensor to the last 2 words of the this_line
    # temp_p = torch.zeros_like(hidden_state)
    # for n in range(B):
    #   seq = this_line_ids[n]
    #   for i in range(M):
    #     if M>2:
    #       if seq[i] == self.s_tok.pad_token_id:
    #         temp_p[n,i-2] = p_hidden[n,0]
    #         temp_p[n,i-1] = p_hidden[n,1]
    

    ##Adding the features together
    # hidden_state = torch.add(hidden_state,temp_p)
    hidden_state = torch.add(hidden_state,k_hidden_cls)
    # hidden_state = hidden_state[None,:,:,:]
    print('hidden' , hidden_state)
    ##Feeding the hidden states into the decoder
    outputs = self.bert2gpt2(encoder_outputs = (hidden_state,), attention_mask = s_mask,
                        decoder_input_ids = input_ids, decoder_attention_mask = d_mask,return_dict = True)
    # outputs = self.bert2gpt2(input_ids=this_line_ids,attention_mask=s_mask, 
    #                     decoder_input_ids = input_ids, decoder_attention_mask = d_mask)

    return outputs
    









##Initilize model

In [ ]:
model = Encoder_Decoder_LM(k_encoder,bert2gpt2,sentence_encoder_tokenizer,keyword_encoder_tokenizer,decoder_tokenizer)
model.to(device)

Encoder_Decoder_LM(
  (w_k): Linear(in_features=768, out_features=768, bias=True)
  (k_encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
       

In [ ]:
# # # load model from epoch 19 from your Google drive
model_path = "./no_phoneme_Bert2GPT/16 ppl 41.8/pytorch_model.bin"

# # # read state dict (each parameter has a name that maps to a tensor)
state_dict = torch.load(model_path, map_location='cpu')

# # # load weights into model
model.load_state_dict(state_dict,strict=False)
# # #print(state_dict)

<All keys matched successfully>

#Training



##Train set up

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup


gradient_accumulation_steps = 1
learning_rate = 1e-5
adam_epsilon = 1e-8
max_grad_norm = 1.0
num_train_epochs = 10
warmup_steps = 0

seed = 42
batch_size = 8
max_seq_length = 139
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

# Loss and Optimizer
t_total = len(train_dataloader) // gradient_accumulation_steps * num_train_epochs

criterion = torch.nn.CrossEntropyLoss(ignore_index = decoder_tokenizer.pad_token_id)
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) 
optimizer = AdamW(model.parameters(), lr=learning_rate, eps=adam_epsilon)
scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps, num_training_steps=t_total
)
print(learning_rate)

1e-05


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


##Model on training


In [ ]:
import os
from tqdm import tqdm

output_root = "./no_phoneme_Bert2GPT"
num_train_epochs = 4
starting_epoch = 19
# Start training
model.train()
global_step = 0
for epoch in tqdm(range(num_train_epochs)):
  tr_loss = 0.0
  last_tr_loss = 0.0
  print("epoch:", epoch+starting_epoch)
  local_step = 0
  for step, batch in enumerate(train_dataloader):
    global_step += 1
    local_step += 1

    # get context and target label
    input_ids, lm_labels, phoneme_ids, keyword_ids, this_line_ids = batch

    # zero grad
    optimizer.zero_grad()

    # print(encoder_attention_mask)
    # print(decoder_attention_mask)
    # logits
    outputs = model(this_line_ids,keyword_ids,input_ids,phoneme_ids)

    # get shapes
    logits = outputs[0]
    B, N, V = logits.size()
    # print('ll' ,logits.size() ,  lm_labels.size())
    # TODO: calculate sum loss.
    # Hint: flatten logits and lm_labels into 2-D tensor and 1-D tensor respectively.
    # i.e. (B*N, V) for the predicts, labels (B*N)
    # 1 line of code
    loss = criterion(logits.view(B*N, V), lm_labels.view(B*N))

    # accumulate total lossa
    tr_loss += loss.item()

    # backprop
    loss.backward()

    # apply gradient clipping
    torch.nn.utils.clip_grad_norm_(bert2gpt2.parameters(), max_grad_norm)

    # model update
    optimizer.step()
    scheduler.step()

    # report performance
    # if (step+1) % (batch_size*4) == 0:
    #   # write avg train loss of current batch to tensorboard
    #   writer.add_scalar("Loss/train", loss.item(), global_step)

    
    if local_step % 800 == 0 and local_step != 0:
      added_loss = tr_loss - last_tr_loss
      last_tr_loss = tr_loss
      print("Average loss over Step", local_step-800, "~ Step",local_step,":", added_loss/800)

  print('loss for epoch {}'.format(epoch+starting_epoch),tr_loss)
  # save model
  state_dict = model.state_dict()
  output_dir = output_root + "/" + str(epoch+starting_epoch)
  os.makedirs(output_dir, exist_ok=True)
  torch.save(state_dict, os.path.join(output_dir, "pytorch_model.bin"))
  print("----------------------------------------------------------------")

# Evaluation

## Perplexity

In [ ]:
import math

def perplexity(iter):
  total_loss = 0.0
  total_tokens = 0
  bert2gpt2.eval()
  criterion1 = torch.nn.CrossEntropyLoss(ignore_index = decoder_tokenizer.pad_token_id, reduction='sum')

  with torch.no_grad():
    for batch in iter:
      # get context and target label
      input_ids, lm_labels, phoneme_ids, keyword_ids, this_line_ids = batch

      # TODO: obtain the valid positions in minibatch to calculate valid number 
      # of tokens
      # 1-2 lines of code
      #print(lm_lables)
      num_tokens = torch.sum(lm_labels != decoder_tokenizer.pad_token_id)

      outputs = model(this_line_ids,keyword_ids,input_ids,phoneme_ids)

      # get shapes
      logits = outputs[0]



      # get shapes
      B, N, V = logits.size()

      # TODO: calculate sum loss. Notice criterion1 is using 'sum' instead of 'mean'
      # Hint: flatten logits and lm_labels into 2-D tensor and 1-D tensor respectively.
      # i.e. (B*N, V) for the predicts, labels (B*N)
      # 1 line of code
      loss = criterion1(logits.view(B*N, V), lm_labels.view(B*N))

      # accumulate total loss
      total_loss += loss.sum().item()

      # accumulate total number of predicted positions
      total_tokens += num_tokens
  print(total_tokens , total_loss)
  return math.exp(total_loss / total_tokens)

In [ ]:
# valid and test PPL, expected range: 50-60
#valid_ppl = perplexity(valid_dataloader)
valid_ppl = perplexity(valid_dataloader)
test_ppl = perplexity(test_dataloader)
#print(valid_ppl)
print(valid_ppl)
print(test_ppl)

hidden tensor([[[ 0.8959,  0.2142, -0.6306,  ...,  1.0976, -1.0475,  0.9246],
         [ 0.8967,  0.2143, -0.6308,  ...,  1.0968, -1.0477,  0.9250],
         [ 0.8956,  0.2145, -0.6309,  ...,  1.0972, -1.0478,  0.9245],
         ...,
         [ 0.8957,  0.2146, -0.6309,  ...,  1.0973, -1.0478,  0.9245],
         [ 0.8957,  0.2145, -0.6309,  ...,  1.0972, -1.0478,  0.9245],
         [ 0.8956,  0.2146, -0.6309,  ...,  1.0973, -1.0478,  0.9245]],

        [[ 0.1614,  0.4972, -0.3261,  ...,  1.0073, -0.5150,  0.8894],
         [ 0.1594,  0.4971, -0.3257,  ...,  1.0084, -0.5147,  0.8887],
         [ 0.1624,  0.4979, -0.3274,  ...,  1.0065, -0.5143,  0.8903],
         ...,
         [ 0.1601,  0.4969, -0.3264,  ...,  1.0076, -0.5143,  0.8893],
         [ 0.1531,  0.5038, -0.3342,  ...,  1.0090, -0.5024,  0.8948],
         [ 0.1608,  0.4977, -0.3275,  ...,  1.0077, -0.5140,  0.8898]],

        [[ 0.4728,  0.8108,  0.0750,  ...,  0.5671, -0.8998,  1.2157],
         [ 0.4735,  0.8110,  0.0741,  

TypeError: ignored

In [ ]:
print(decoder_tokenizer.pad_token_id)

#Generator


In [ ]:
import nltk
nltk.download('cmudict')


def sentence_vowel(line,limit=2):
    line = decoder_tokenizer.decode(line)
    l = []
    all_vowels = {"AA":0,"AE":1,"AH":2,"AO":3,"AW":4,"AY":5,"EH":6,"ER":7,"EY":8,"IH":9,"IY":10,"OW":11,"OY":12,"UH":13,"UW":14,"[Empty]":15}
    dic = nltk.corpus.cmudict.dict()
    if line != '[Empty]':
        vowels = []
        words = line.split()[-5:]
        for i in range(len(words)-1,-1,-1):
            if len(vowels)>limit-1:
                break
            try:
                word = words[i].lower()
                count = 0
                while word not in dic.keys() and count < 2:
                    count += 1
                    word = word[:-1]
                sounds = dic[word]
                sounds = sounds[0]
                for j in range(len(sounds)-1,-1,-1):
                    if len(vowels)>limit-1:
                        break
                    if sounds[j] in all_vowels.keys():
                        vowels.append(sounds[j])
                    elif sounds[j][:-1] in all_vowels.keys():
                        vowels.append(sounds[j][:-1])
            except:
                continue
        while len(vowels) < limit:
            vowels.append("[Empty]")



    else:
        vowels = ["[Empty]"]*limit


    vowels.reverse()
    vowel_ids = [all_vowels[vowels[i]] for i in range(len(vowels))]
    vowel_ids = torch.LongTensor(vowel_ids)[None , :].to(device)


    return vowel_ids

print(sentence_vowel(decoder_tokenizer.encode("I fuck your mum")))


[nltk_data] Downloading package cmudict to /content/drive/My
[nltk_data]     Drive/nltk_data...
[nltk_data]   Package cmudict is already up-to-date!
tensor([[3, 2]], device='cuda:0')


In [ ]:
def top_filtering(logits, top_k=0, top_p=0.0, threshold=-float('Inf'), filter_value=-float('Inf')):    
    """ Filter a distribution of logits using top-k, top-p (nucleus) and/or threshold filtering
        Args:
            logits: logits distribution shape (..., vocabulary size)
            top_k: <=0: no filtering, >0: keep only top k tokens with highest probability.
            top_p: <=0.0: no filtering, >0.0: keep only a subset S of candidates, where S is the smallest subset
                whose total probability mass is greater than or equal to the threshold top_p.
                In practice, we select the highest probability tokens whose cumulative probability mass exceeds
                the threshold top_p.
            threshold: a minimal threshold to keep logits
    """
    top_k = min(top_k, logits.size(-1))
    if top_k > 0:
        # Remove all tokens with a probability less than the last token in the top-k tokens
        indices_to_remove = logits < torch.topk(logits, top_k)[0][..., -1, None]
        logits[indices_to_remove] = filter_value

    if top_p > 0.0:
        # Compute cumulative probabilities of sorted tokens
        sorted_logits, sorted_indices = torch.sort(logits, descending=True)
        cumulative_probabilities = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)

        # Remove tokens with cumulative probability above the threshold
        sorted_indices_to_remove = cumulative_probabilities > top_p
        # Shift the indices to the right to keep also the first token above the threshold
        sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
        sorted_indices_to_remove[..., 0] = 0

        # Back to unsorted indices and set them to -infinity
        indices_to_remove = sorted_indices[sorted_indices_to_remove]
        logits[indices_to_remove] = filter_value

    indices_to_remove = logits < threshold
    logits[indices_to_remove] = filter_value

    return logits

In [ ]:
def count_tri(tri , sentence):
  count = 0
  l = len(tri)
  for i in range(len(sentence) - (l-1)):
    my_tri = sentence[i:i+l]
    if my_tri == tri:
      count += 1
  return count

In [ ]:
import math
def phoneme_score(p1 , p2):
  p1 = p1.float()
  p2 = p2.float()
  return math.log(torch.dot(p1.squeeze() , p2.squeeze()).item())

In [ ]:
from torch import nn
import random as rd
def beam_search(decoder , this_line_ids , keyword , phoneme, beam=50 , min_len=15 , max_len = 30  , last_tri = None):
    k = beam
    k_prev_words = torch.full((k, 1), decoder_tokenizer.bos_token_id , dtype=torch.long).to(device) # (k, 1)
    seqs = k_prev_words #(k, 1)
    # if second:
    #   kk = torch.full((k, 1), f_word , dtype=torch.long).to(device)
    #   seqs = torch.cat((k_prev_words,kk),1)
    top_k_scores = torch.zeros(k, 1).to(device)
    complete_seqs = []
    complete_seqs_scores = []
    incomplete_seqs = []
    incomplete_seqs_scores = []
    step = 1
    seqs_scores = torch.zeros(beam).to(device)
    vocab_size = decoder_tokenizer.vocab_size
    the = 5e-4
    while True:
        print(step , seqs.size())
        print('depth {}'.format(step),seqs)
        outputs = model(this_line_ids,keyword,seqs,phoneme)[0]  # outputs: (k, seq_len, vocab_size)
        outputs = top_filtering(outputs / 0.8)
        next_token_logits = nn.Softmax(dim=-1)(outputs)[:,-1,:] # (k, vocab_size)
        print('next',next_token_logits.size())
        for i in range(k):
          ls = seqs[i][1:]
          for ele in ls:
            next_token_logits[i][ele] = 0
        if step == 1:
        
          top_k_scores, top_k_words = next_token_logits[0].topk(k, dim=0, largest=True, sorted=True)
        else:
        
          top_k_scores, top_k_words = next_token_logits.contiguous().view(-1).topk(k, 0, True, True)
        if step >= min_len:
          f_logits =  next_token_logits.contiguous().view(-1)
          for i in range(k):
            if f_logits[i * decoder_tokenizer.eos_token_id] > the:
              top_k_words[i] = decoder_tokenizer.eos_token_id

        # seqs_scores += top_k_scores
        prev_word_inds = top_k_words // vocab_size  # (k)  beam_id
        next_word_inds = top_k_words % vocab_size  # (k)  token_id
        print('prev word' , prev_word_inds)
        print('topk {}'.format(step) , next_word_inds)
        print('topk scores{}'.format(step) , top_k_scores)
        # seqs: (k, step) ==> (k, step+1)
        seqs = torch.cat([seqs[prev_word_inds], next_word_inds.unsqueeze(1)], dim=1)
        # print('sss',seqs)
        # unfinished sentence
        incomplete_inds = [ind for ind, next_word in enumerate(next_word_inds) if
                                next_word != decoder_tokenizer.eos_token_id]
        # finished sentence
        complete_inds = list(set(range(len(next_word_inds))) - set(incomplete_inds))

        if len(complete_inds) > 0:
            complete_seqs.extend(seqs[complete_inds].tolist()) 
            complete_seqs_scores.extend(top_k_scores[complete_inds]) 
        # if len(incomplete_inds) > 0):
        #     incomplete_seqs.extend(seqs[incomplete_inds].tolist()) 
        #     incomplete_seqs_scores.extend(top_k_scores[incomplete_inds] 
        k -= len(complete_inds) 
  
        if k == 0: # finished
           break
  
        # updating 
        seqs = seqs[incomplete_inds]
        # hidden = hidden[prev_word_inds[incomplete_inds]]
        top_k_scores = top_k_scores[incomplete_inds]   #(s, 1) s < k
        top_k_scores = top_k_scores.unsqueeze(1)
        k_prev_words = next_word_inds[incomplete_inds] #(s, 1) s < k
        k_prev_words = k_prev_words.unsqueeze(1)
        if step > max_len: #too long
            break
        step += 1
    
    ## Prune Trigram path
    for i in range(len(complete_seqs_scores)):
      if last_tri is not None and count_tri([46, 1860, 10898, 318] , complete_seqs[i]) >= 1:
        complete_seqs_scores[i] = 0

    ## Integrate beam_search_scores with phoneme_score
    final_score = []
    for i in range(len(complete_seqs_scores)):
      list_sen = complete_seqs[i]
      p1 = sentence_vowel(list_sen)
      p2 = phoneme
      print('ppppp' ,p1)
      print('ppppp' ,p2)
      p_score = phoneme_score(p1 ,p2)
      print('ppppp' ,p_score)
      beam_score = complete_seqs_scores[i]
      final_score.append(beam_score + 0.08*p_score)
    try:
      i = final_score.index(max(final_score)) # max score sequence
      seq = torch.LongTensor(complete_seqs[i])
    except BaseException:
      # value, i = seqs_scores.topk(1, 0, True, True)
      i = rd.randint(0,9)
      seq = torch.LongTensor(seqs[i].to('cpu'))
    return seq[None,:].to(device)

In [ ]:
import random as rd
def FS():
  target = rd.randint(1,100)
  for step, batch in enumerate(train_dataloader):
    if step == target:
      input_ids, lm_labels, phoneme_ids, keyword_ids, this_line_ids = batch
      sentence = this_line_ids[1]
      keyword = keyword_ids[1]
      target = input_ids[1]
      sentence = sentence[sentence.nonzero()].squeeze()
      keyword = keyword[keyword.nonzero()].squeeze()
      return sentence[None,:].to(device) , keyword.to(device) , target.to(device)

In [ ]:
tri_1 = "I'm sorry"
tri_2 = 'You know what'
tri_3 = decoder_tokenizer.encode("I've been")
tri_1 = decoder_tokenizer.encode(tri_1)
tri_2 = decoder_tokenizer.encode(tri_2)
print(tri_1)

[40, 1101, 7926]


In [ ]:
def generator(model , keyword):
  lyrics = []
  first_line ,keyword1 , target= FS()
  lyrics.append(first_line)
  last_tri = None
  phoneme = None
  keyword = keyword_encoder_tokenizer.encode(keyword)
  keyword = torch.LongTensor(keyword)[None,:].to(device)

  print('sentence 0',first_line)
  print('target' , decoder_tokenizer.decode(target))
  print('target_id',target)
  for i in range(1,4):
    print('iiii',i)
    last_line = lyrics[i-1]
    if i > 1:
      last_line = last_line[:,1:].squeeze()

      if count_tri(tri_1 , last_line) >= 1:
        last_tri = tri_1
      # elif count_tri(tri_2 , last_line) >= 1:
      #   last_tri = tri_2
      
      string = decoder_tokenizer.decode(last_line)
      last_line = sentence_encoder_tokenizer.encode(string)
      phoneme = sentence_vowel(last_line)
      last_line = torch.LongTensor(last_line)[None,:].to(device)
    else:
      s_l_line = last_line.squeeze()
      phoneme = sentence_vowel(s_l_line)
    next_line = beam_search(model, last_line , keyword , phoneme , last_tri=last_tri )
    lyrics.append(next_line)
  for i in range(len(lyrics)):
    sentence = lyrics[i]
    if i == 0:
      print(sentence_encoder_tokenizer.decode(sentence.squeeze()))
    else:
      print(decoder_tokenizer.decode(sentence.squeeze()))
  return lyrics

In [ ]:
res = generator(model ,keyword = 'religion , race , gangster')

sentence 0 tensor([[ 101, 1045, 2001, 2196, 6069, 2994, 1996, 2168,  102]],
       device='cuda:0')
target <|endoftext|>All I ever really wanted was to change the game<|endoftext|><|endoftext|><|endoftext|><|endoftext|>
target_id tensor([50256,  3237,   314,  1683,  1107,  2227,   373,   284,  1487,   262,
          983, 50256, 50256, 50256, 50256], device='cuda:0')
iiii 1
1 torch.Size([50, 1])
depth 1 tensor([[50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').


流式输出内容被截断，只能显示最后 5000 行内容。
9 torch.Size([50, 9])
depth 9 tensor([[50256,    45,   328,  4908,    11,   345, 18959,   447,   247],
        [50256,    44,  1436,   286,  1109,    11,   314,   447,   247],
        [50256,    47,  2433,   284,  1793,    11,   314,   447,   247],
        [50256,    19,   400,  9559,    11,   314,   447,   247,    76],
        [50256,    45,   328,  4908,    11,   314,   447,   247,    76],
        [50256,    42,   747,   502,    11,   314,   447,   247,    76],
        [50256,    44,  2002,    64,    11,   314,   447,   247,    76],
        [50256,    47,  2433,   284,  1793,   329,   262,   661,   326],
        [50256,    26,   314,   447,   247,    76,   257,   582,   286],
        [50256,    11,   314,   447,   247,    76,   257,   582,   286],
        [50256,    16,   301,   290,  1315,   400, 10675,  2084,    11],
        [50256,    43,   452,   259,     6,   262,   995,   257,  6486],
        [50256,    44,  2002,    64,    11,   314,  1101,   257,  2

In [ ]:
tri_3 = decoder_tokenizer.encode("Odd Future is")
tri_4 = decoder_tokenizer.encode("Odd Future is")


In [ ]:
def generator_wf(model , keyword , f_s):
  lyrics = []
  lyrics.append(f_s)
  last_tri = None
  phoneme = None
  keyword = keyword_encoder_tokenizer.encode(keyword)
  keyword = torch.LongTensor(keyword)[None,:].to(device)
  for i in range(1,4):
    print('iiii',i)
    last_line = lyrics[i-1]
    last_line = last_line[:,1:].squeeze()

    if count_tri(tri_1 , last_line) >= 1:
      last_tri = tri_3
    # elif count_tri(tri_2 , last_line) >= 1:
    #   last_tri = tri_2
      
    string = decoder_tokenizer.decode(last_line)
    last_line = sentence_encoder_tokenizer.encode(string)
    phoneme = sentence_vowel(last_line)
    last_line = torch.LongTensor(last_line)[None,:].to(device)
    
    next_line = beam_search(model, last_line , keyword , phoneme , last_tri=last_tri )
    lyrics.append(next_line)
  for i in range(len(lyrics)):
    sentence = lyrics[i]
    print(decoder_tokenizer.decode(sentence.squeeze()))
  return lyrics

In [ ]:
res1 = generator_wf(model , 'religion , race , gangster' , f_s=res[-1])

iiii 1
1 torch.Size([50, 1])
depth 1 tensor([[50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256],
        [50256]], device='cuda:0')
hidden tensor([[[ 0.2117,  0.4020, -0.4399,  ...,  0.8197, -0.4950,  0.5803],
         [ 0.2124

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').


流式输出内容被截断，只能显示最后 5000 行内容。
        [50256,    33,  2007,    11,   345,   760,   644,   314,  1101,  2282],
        [50256,     6, 42323,   314,  1101,   257,  2802,    69, 12603,    11],
        [50256,    46,  1860, 10898,   318,   262,   691,   835,   345,   460],
        [50256,    11,   314,   447,   247,    76,   257,   582,   286,   616],
        [50256,     1,    40,   836,   470,   760,   644,   345,   892,   546],
        [50256,    42,   747,   502,    11,   314,   447,   247,    76,   257],
        [50256,    33,  2007,    11,   314,   447,   247,    76,   407,   257],
        [50256,    16,    13,    17,  1510,   661,   287,   262,   717,  1285],
        [50256,     1,    40,   836,   470,   765,   345,   284,   766,   502],
        [50256,     1,    40,   836,   470,   760,   644,   345,   821,  2282],
        [50256,    44,  1689,    11,   314,  1101,  2611,   307,   257,  2802],
        [50256,    46,  1860, 10898,   318,   262,   691,   835,   356,   460],
        [5025